In [1]:
import math
import mip
from collections import namedtuple

In [2]:
##### Define the number of games in the ticket plan #####

GamesInPlan = 81

#########################################################

# Build a dictionary out of a list of games
#
#     buildCode == 0:    Create constraints for pair and quads
#     buildCode == 2:    Create constraint for pairs only
#     buildCode == 4:    Create constraint for quads only

def BuildDict(gameList, buildCode = 0):
    pairList = []
    for game in gameList:
        if buildCode != 4:
            pairList.append((game, 1.0))
        if buildCode != 2:
            pairList.append((game + GamesInPlan, 1.0))
    return dict(pairList)

# Read the season schedule and store it as a list of Games

Game = namedtuple('Game', ('month', 'day', 'gameDay', 'weekday', 'opponent', 'time', 'costCode'))
        
def Schedule(scheduleFile):
    file = open(scheduleFile, "r")
    schedule = file.readlines()
    monthDays = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    cumulative = []
    sofar = 0
    games = []
    for days in monthDays:
        sofar += days
        cumulative.append(sofar)
    for gix, game in enumerate(schedule):
        if gix >= GamesInPlan:
            continue
        fields = game.split()
        date = fields[0].split("/")
        month = int(date[0])
        day = int(date[1])
        gameDay = cumulative[month - 2] + day
        games.append(Game(month, day, gameDay, fields[1], fields[2], fields[3], fields[4]))
    return games

# Load in the schedule

schedule = Schedule("c:/users/thoma/tom/baseball/2025/sched.2025")

# Define a class to contain constraints

class Constraint:
    def __init__(self, type, rhs, coefDict):
        if type == '=':
            self.type = 'E'
        if type == '<':
            self.type = 'L'
        if type == '>':
            self.type = 'G'
        self.rhs = rhs
        self.coefDict = coefDict

# Define a class to contain each person's preferences

class MarinersFan:
    def __init__(self, name, pairs, quads, ranking, extra):
        self.name = name
        self.pairs = pairs
        self.quads = quads

# Assign weights to games

        if len(ranking) != 0:
            self.ranking = ranking
        else:
            self.ranking = GamesInPlan * [GamesInPlan // 2]

# Adjust weights to favor highly ranked games

        self.useRanking = []
        midpoint = (GamesInPlan - 1) // 2
        for wgt in self.ranking:
            if wgt <= midpoint + 1:
                self.useRanking.append(math.sqrt(wgt - 1.0))
            else:
                self.useRanking.append(2.0 * math.sqrt(midpoint) - math.sqrt(2.0 * midpoint - wgt + 1))
        if len(self.useRanking) == GamesInPlan:
            self.useRanking += [2.0 * cost for cost in self.useRanking]
        else:
            for ix in range(GamesInPlan):
                self.useRanking[GamesInPlan + ix] *= 2

# Each person must attend correct number of games

        pairCon = Constraint('=', self.pairs, dict([(ix, 1.0) for ix in range(GamesInPlan)]))
        quadCon = Constraint('=', self.quads, dict([(ix, 1.0) for ix in range(GamesInPlan, 2 * GamesInPlan)]))

# Save all of the constraints for this person

        self.constraints = [pairCon, quadCon] + extra

##### This constraint handles the spacing of games

def Spacing(daysApart, comparator, value, pairsOrQuads = 0):
    firstGame = 0
    lastGame = 0
    constraintList = []
    while True:
        while lastGame < GamesInPlan and schedule[firstGame].gameDay + daysApart > schedule[lastGame].gameDay:
            lastGame += 1
        constraintList.append(Constraint(comparator, value, BuildDict(range(firstGame, lastGame), pairsOrQuads)))
        if lastGame == GamesInPlan:
            break
        while schedule[firstGame].gameDay + daysApart <= schedule[lastGame].gameDay:
            firstGame += 1
    return constraintList

##### Require or forbid games in various months

def Monthly(comparator, value, pairsOrQuads = 0):
    firstGame = 0
    lastGame = 0
    constraintList = []
    while True:
        while schedule[lastGame].month < 5:
            lastGame += 1
        while lastGame < GamesInPlan and schedule[firstGame].month == schedule[lastGame].month:
            lastGame += 1
        constraintList.append(Constraint(comparator, value, BuildDict(range(firstGame, lastGame), pairsOrQuads)))
        if lastGame == GamesInPlan:
            break
        firstGame = lastGame
    return constraintList

##### Require or forbid games in different series

def Series(comparator, value, pairsOrQuads = 0):
    firstGame = 0
    lastGame = 0
    constraintList = []
    while True:
        while lastGame < GamesInPlan and schedule[firstGame].opponent == schedule[lastGame].opponent:
            lastGame += 1
        constraintList.append(Constraint(comparator, value, BuildDict(range(firstGame, lastGame), pairsOrQuads)))
        if lastGame == GamesInPlan:
            break
        firstGame = lastGame
    return constraintList

##### Define the extra constraints right here #####

def NoAprilGames():
    return [Constraint ('=', 0.0, BuildDict(range (0, 18)))]
def NoSundayGames():
    return [Constraint ('=', 0.0, BuildDict([3, 12, 15, 20, 29, 35, 45, 48, 55, 61, 64, 74, 80]))]
def NoWeekdayDayGames():
    return [Constraint ('=', 0.0, BuildDict([6, 9, 17, 23, 32, 38, 51, 58, 67]))]
def Weekend_Quads_Only():
    return [Constraint ('=', 0.0, BuildDict([0, 4, 5, 6, 7, 8, 9, 10, 16, 17, 21, 22, 23, 24,
                                             25, 26, 30, 31, 32, 36, 37, 38, 39, 40, 41, 42,
                                             49, 50, 51, 52, 56, 57, 58, 65, 66, 67, 68, 69, 70,
                                             71, 75, 76, 77], 4))]
def Blackout(games):
    return [Constraint('=', 0.0, BuildDict(games))]
def Special_AE():
    return [Constraint('>', 1.0, BuildDict([50, 51, 52], 4))]


|               | Pairs | Quads | Total Pairs |
| :------------ | ----: | ----: | ----------: |
| Karen Boehmer | 1 | 4 | 9 |
| Jordan Bork   | 4 | 3 | 10 |
| Eric Brechner | 22 | | 22 |
| Al Erisman |14 | 1 | 16 |
| Andrew Erisman | | 9 | 18 |
| Michael Erisman | 6 | 3 | 12 |
| Ryan Falls | | 4 | 8 |
| Tom Grandine | 10 | 2 | 14 |
| Cole Graves | 4 | 5 | 14 |
| Michael Jones | 4 | | 4 |
| Fritz Klein | 8 | 2 | 12 |
| Nick Lederer | 4 | | 4 |
| Mike Neff | 6 | 4 | 14 |
| Mike Rochester | 5 | | 5 |
| | | **Total** | 162 |            162


In [3]:
##### Define the participants here #####

fans = [MarinersFan('Karen Boehmer   ', 1, 4, [31, 63, 64, 40, 41, 36, 62, 42, 37, 9, 45, 30, 39, 43, 61, 29, 47,
                                               7, 35, 38, 17, 65, 3, 1, 18, 55, 44, 34, 54, 8, 58, 28, 66, 59, 23,
                                               67, 15, 68, 2, 69, 70, 71, 72, 73, 74, 75, 60, 22, 4, 19, 76, 46, 77,
                                               14, 5, 56, 78, 16, 51, 20, 10, 6, 13, 11, 52, 12, 21, 50, 33, 53, 27,
                                               79, 57, 26, 80, 25, 32, 48, 24, 81, 49],
                    Blackout([2, 21, 32, 35, 37, 39, 40, 41, 42, 43, 44, 45, 50, 52, 56, 71, 74, 79]) + Series('<', 1.0)),
        MarinersFan('Jordan Bork     ', 4, 3, [13, 81, 80, 79, 31, 78, 77, 3, 76, 75, 4, 74, 73, 72, 10, 1, 15, 22, 7,
                                               23, 71, 29, 17, 27, 26, 25, 24, 70, 12, 2, 69, 68, 67, 28, 11, 19, 66,
                                               65, 64, 63, 62, 61, 60, 59, 16, 5, 58, 57, 6, 56, 55, 54, 53, 14, 8, 9,
                                               52, 51, 50, 36, 32, 30, 18, 20, 21, 33, 49, 48, 47, 46, 45, 44, 43, 42,
                                               41, 40, 39, 38, 34, 35, 37,
                                               40, 39, 38, 37, 41, 42, 43, 44, 45, 46, 3, 36, 35, 34, 9, 1, 47, 48, 6,
                                               18, 33, 49, 50, 51, 52, 53, 54, 32, 11, 2, 55, 56, 57, 19, 10, 15, 58,
                                               59, 60, 61, 62, 63, 64, 31, 13, 4, 30, 29, 5, 65, 66, 67, 68, 12, 7, 8,
                                               69, 70, 71, 24, 21, 20, 14, 16, 17, 72, 73, 74, 75, 76, 77, 78, 28, 27,
                                               26, 79, 80, 81, 22, 23, 25],
                    Blackout([1, 2, 3, 5, 6, 8, 9, 11, 12, 13, 20, 27, 30, 31, 32, 36, 37, 38, 39, 40, 41, 42, 43, 46,
                              47, 49, 50, 51, 52, 56, 57, 58, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]) +
                    Monthly('<', 2.0) + Series('<', 1.0) + Weekend_Quads_Only()),
        MarinersFan('Eric Brechner   ', 22, 0, [36, 11, 81, 80, 69, 24, 49, 68, 62, 61, 35, 34, 33, 23, 22, 21, 32, 48,
                                                45, 44, 43, 42, 41, 47, 3, 46, 2, 67, 60, 59, 66, 58, 57, 7, 20, 19, 40,
                                                39, 38, 37, 79, 78, 77, 76, 75, 74, 6, 31, 30, 56, 55, 65, 73, 72, 71,
                                                70, 54, 64, 53, 51, 1, 50, 5, 29, 28, 10, 18, 17, 16, 15, 9, 27, 4, 26,
                                                25, 63, 52, 8, 14, 13, 12],
                     Series('<', 1.0)),
        MarinersFan('Al Erisman      ', 14, 1, [4, 78, 79, 80, 17, 18, 81, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,
                                                74, 75, 76, 77, 19, 20, 56, 57, 58, 59, 60, 61, 24, 25, 26, 40, 41, 5,
                                                6, 7, 8, 21, 22, 23, 27, 62, 9, 10, 28, 29, 30, 1, 2, 3, 42, 43, 44, 45,
                                                31, 32, 33, 49, 50, 51, 46, 47, 48, 34, 35, 36, 14, 15, 16, 52, 53, 54,
                                                55, 37, 38, 39, 11, 12, 13],
                    Spacing(2, '<', 1.0) + Series('<', 1.0) + Special_AE()),
        MarinersFan('Andrew Erisman  ', 0, 9, [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
                                               50, 50, 50, 50, 50, 50, 50, 3, 50, 50, 4, 50, 50, 5, 50, 50, 6, 50, 1,
                                               50, 2, 50, 50, 7, 50, 50, 50, 50, 8, 50, 50, 9, 50, 50, 10, 50, 50, 11,
                                               50, 50, 12, 50, 50, 13, 50, 50, 14, 50, 50, 15, 50, 50, 16, 50, 50, 17,
                                               50, 50, 18, 50, 19, 50, 20],
                    Monthly('<', 2.0)),
        MarinersFan('Michael Erisman ', 6, 3, GamesInPlan * [40],
                    Spacing(2, '<', 1.0) + Monthly('<', 2.0) + Series('<', 1.0)),
        MarinersFan('Ryan Falls      ', 0, 4, [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 11, 12, 34, 13, 14, 35, 36, 37,
                                               38, 39, 40, 7, 8, 41, 42, 43, 10, 15, 44, 45, 5, 6, 46, 47, 48, 22, 1,
                                               2, 49, 50, 51, 52, 53, 54, 9, 55, 3, 4, 56, 18, 19, 57, 23, 58, 59, 60,
                                               61, 62, 63, 64, 65, 66, 20, 21, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,
                                               77, 78, 79, 80, 16, 17, 81],
                    Spacing(14, '<', 1.0) + Monthly('<', 1.0) + Blackout([4])),
        MarinersFan('Tom Grandine    ', 10, 2, [1, 10, 11, 52, 50, 46, 53, 47, 48, 54, 9, 8, 55, 7, 49, 56, 72, 73, 37,
                                                38, 57, 39, 40, 58, 41, 42, 43, 44, 45, 59, 74, 33, 75, 34, 6, 60, 35,
                                                36, 61, 19, 20, 21, 22, 62, 5, 63, 23, 24, 64, 25, 26, 27, 76, 77, 78,
                                                79, 28, 29, 65, 4, 30, 66, 31, 32, 67, 18, 3, 68, 80, 81, 51, 69, 17,
                                                16, 70, 15, 14, 13, 2, 12, 71],
                    Spacing(2, '<', 1.0) + Monthly('>', 1.0) + Series('<', 1.0) + NoWeekdayDayGames() +
                    Blackout([5, 6, 16, 17, 30, 32, 52, 53, 54, 55, 69])),
        MarinersFan('Cole Graves     ', 4, 5, [17, 18, 56, 57, 6, 24, 25, 2, 29, 30, 19, 58, 59, 26, 60, 61, 27, 28, 20,
                                               62, 63, 8, 31, 32, 33, 34, 16, 10, 64, 65, 37, 38, 39, 23, 66, 67, 11, 35,
                                               36, 4, 40, 41, 12, 21, 68, 69, 9, 70, 71, 1, 42, 43, 15, 3, 72, 73, 46, 47,
                                               54, 55, 74, 75, 14, 76, 77, 5, 44, 45, 7, 53, 48, 49, 50, 78, 79, 51, 52, 22,
                                               13, 80, 81, 27, 28, 19, 15, 29, 30, 31, 32, 33, 34, 35, 5, 11, 36, 20, 21,
                                               37, 38, 39, 18, 17, 40, 41, 42, 43, 44, 45, 46, 22, 4, 47, 48, 49, 50, 2, 14,
                                               51, 52, 53, 54, 55, 56, 57, 58, 8, 7, 59, 3, 13, 60, 61, 62, 63, 64, 23, 9,
                                               65, 66, 67, 68, 26, 10, 69, 12, 16, 70, 71, 72, 73, 74, 75, 76, 77, 24, 25,
                                               78, 79, 80, 81, 1, 6],
                    Spacing(2, '<', 1.0) + Series('<', 1.0) + Weekend_Quads_Only() +
                    Blackout([2, 5, 6, 8, 9, 13, 14, 15, 16, 17, 19, 22, 23, 24, 25, 28, 30, 31, 32, 37, 38, 40, 41, 50, 51,
                              54, 56, 57, 58, 59, 60, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76])),
        MarinersFan('Michael Jones   ', 4, 0, [31, 22, 23, 71, 59, 52, 42, 58, 51, 41, 24, 25, 70, 26, 27, 69, 50, 40, 9,
                                               72, 68, 56, 48, 39, 46, 36, 18, 8, 16, 67, 44, 35, 30, 3, 12, 66, 54, 43,
                                               38, 55, 45, 34, 10, 1, 11, 65, 2, 20, 64, 57, 47, 37, 21, 4, 28, 63, 49,
                                               33, 29, 5, 19, 62, 73, 74, 75, 76, 77, 78, 79, 80, 81, 17, 6, 13, 61, 53, 
                                               32, 14, 7, 15, 60],
                    Spacing(10, '<', 1.0) + NoAprilGames() + NoSundayGames() + NoWeekdayDayGames() +
                    Blackout([19, 62, 63, 64, 65, 66, 67, 68, 69, 70])),
        MarinersFan('Fritz Klein     ', 8, 2, [78, 77, 76, 75, 74, 73, 10, 64, 63, 9, 62, 61, 60, 59, 58, 57, 56, 8, 70,
                                               71, 72, 31, 30, 7, 29, 5, 28, 34, 35, 36, 18, 27, 1, 55, 54, 53, 33, 32, 6,
                                               21, 26, 14, 79, 66, 65, 37, 52, 51, 50, 24, 20, 2, 17, 67, 69, 68, 13, 23,
                                               3, 49, 48, 47, 46, 45, 44, 22, 19, 4, 25, 11, 80, 16, 43, 42, 41, 12, 81,
                                               15, 40, 39, 38, 78, 77, 76, 75, 74, 73, 62, 64, 63, 79, 65, 61, 60, 59, 58,
                                               57, 56, 35, 70, 71, 72, 31, 30, 36, 29, 41, 28, 34, 8, 7, 55, 54, 50, 18, 27,
                                               3, 33, 32, 6, 21, 26, 14, 25, 66, 40, 37, 52, 51, 1, 24, 9, 44, 17, 67, 69,
                                               68, 13, 23, 53, 49, 48, 47, 46, 45, 2, 22, 19, 38, 20, 11, 80, 16, 43, 42,
                                               5, 12, 81, 15, 10, 39, 4],
                    Spacing(10, '<', 1.0, 2) + Spacing(28, '>', 1.0, 2) + Spacing(21, '<', 1.0, 4) + Spacing(7, '<', 1.0)),
        MarinersFan('Nick Lederer    ', 4, 0, [58, 1, 5, 29, 30, 56, 54, 53, 52, 51, 31, 71, 72, 70, 69, 68, 67, 32, 7,
                                               3, 15, 39, 38, 37, 36, 35, 34, 17, 18, 50, 65, 66, 73, 49, 19, 20, 74, 75,
                                               76, 21, 48, 11, 46, 22, 23, 24, 6, 2, 9, 47, 45, 44, 43, 25, 16, 26, 42,
                                               41, 40, 27, 28, 14, 33, 77, 78, 79, 80, 81, 64, 63, 62, 61, 60, 13, 12,
                                               59, 57, 55, 4, 8, 10],
                    Blackout([11, 12, 32, 36, 37, 38, 63, 64, 65, 66, 67]) + Monthly('<', 1.0)),
        MarinersFan('Mike Neff       ', 6, 4, [40, 39, 74, 38, 62, 61, 77, 55, 52, 76, 27, 19, 23, 28, 25, 26, 58, 78, 37,
                                               36, 35, 3, 2, 4, 66, 65, 64, 47, 46, 45, 63, 79, 81, 34, 18, 22, 49, 48, 75,
                                               54, 53, 51, 24, 16, 12, 20, 33, 5, 8, 60, 59, 80, 32, 31, 13, 21, 71, 70, 73,
                                               30, 1, 29, 10, 9, 11, 15, 14, 72, 57, 56, 50, 44, 43, 17, 42, 69, 68, 67,
                                               6, 7, 41],
                    Monthly('>', 1.0)),
        MarinersFan('Mike Rochester  ', 5, 0, [81, 80, 79, 78, 60, 59, 58, 40, 39, 23, 8, 38, 37, 77, 76, 75, 57, 56, 5, 4,
                                               3, 36, 35, 22, 55, 54, 53, 74, 73, 72, 52, 51, 50, 2, 17, 7, 34, 33, 21, 32,
                                               31, 30, 29, 19, 16, 15, 71, 70, 69, 49, 48, 47, 68, 67, 66, 65, 46, 45, 44,
                                               6, 14, 13, 1, 12, 11, 28, 27, 20, 43, 42, 41, 64, 63, 62, 61, 26, 25, 24,
                                               18, 10, 9],
                    Spacing(14, '<', 1.0))]

In [4]:
tixmodel = mip.Model()
tixvars = [tixmodel.add_var(var_type = mip.BINARY) for ix in range(2 * GamesInPlan)]
for fan in fans:
    for constraint in fan.constraints:
        linfunc = mip.xsum(constraint.coefDict[ix] * tixvars[ix] for ix in constraint.coefDict)
        if constraint.type == 'E':
            tixmodel += linfunc == constraint.rhs
        if constraint.type == 'L':
            tixmodel += linfunc <= constraint.rhs
        if constraint.type == 'G':
            tixmodel += linfunc >= constraint.rhs

In [5]:
tixmodel = mip.Model()
tixvars = [tixmodel.add_var(var_type = mip.BINARY) for ix in range(2 * GamesInPlan * len(fans))]

# All tickets must be allocated

for ix in range(GamesInPlan):
    tixmodel += mip.xsum(tixvars[ix + 2 * iy * GamesInPlan] + 2.0 * tixvars[ix + GamesInPlan + 2 * iy * GamesInPlan] for iy in range(len(fans))) == 2

# Each fan must attend the correct number of games + satisfy all personal constraints

for iy in range(len(fans)):
    for constraint in fans[iy].constraints:
        linfunc = mip.xsum(constraint.coefDict[ix] * tixvars[ix + iy * 2 * GamesInPlan] for ix in constraint.coefDict)
        if constraint.type == 'E':
            tixmodel += linfunc == constraint.rhs
        if constraint.type == 'L':
            tixmodel += linfunc <= constraint.rhs
        if constraint.type == 'G':
            tixmodel += linfunc >= constraint.rhs

# Establish the objective function

costs = []
for fan in fans:
    costs += fan.useRanking
tixmodel.objective = mip.xsum(costs[ix] * tixvars[ix] for ix in range(len(tixvars)))

In [6]:
tixmodel.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [7]:
gameCost = {'A' : 90, 'B' : 69, 'C' : 57, 'D' : 45}
costs = (len(fans) * [0.0])[:]
print("| | | Date | Day | Opponent | Time | Cost |")
print("| :- | :- | ---: | --- | :-----: | ---: | ---- |")
for gix, game in enumerate(schedule):
    for mix, mvar in enumerate(tixmodel.vars):
        if mix % GamesInPlan == gix and mvar.x > 0.5:
            fix = mix // (2 * GamesInPlan)
            if len(fans[fix].ranking) > GamesInPlan and mix % (2 * GamesInPlan) >= GamesInPlan:
                gix += GamesInPlan
            costs[fix] += 2.0 * gameCost[game.costCode]
            print("| " + fans[fix].name + " (%d)  " % (fans[fix].ranking[gix]), end = '')
            if mix % (2 * GamesInPlan) >= GamesInPlan:
                costs[fix] += 2.0 * gameCost[game.costCode]
                print("|                       ", end = '')
    for fix, fan in enumerate(fans):
        continue
    print("| %d/%d | " % (game.month, game.day) + game.weekday + " | " + game.opponent + " | " + game.time +
          " | " + game.costCode + " |")

| | | Date | Day | Opponent | Time | Cost |
| :- | :- | ---: | --- | :-----: | ---: | ---- |
| Al Erisman       (4)  | Tom Grandine     (1)  | 3/27 | Thu | ATH | 7:10pm | A |
| Eric Brechner    (11)  | Nick Lederer     (1)  | 3/28 | Fri | ATH | 6:40pm | B |
| Michael Erisman  (40)  |                       | 3/29 | Sat | ATH | 6:40pm | B |
| Cole Graves      (15)  |                       | 3/30 | Sun | ATH | 1:10pm | B |
| Al Erisman       (17)  | Tom Grandine     (50)  | 3/31 | Mon | DET | 6:40pm | D |
| Michael Erisman  (40)  |                       | 4/1 | Tue | DET | 6:40pm | D |
| Eric Brechner    (49)  | Fritz Klein      (10)  | 4/2 | Wed | DET | 1:10pm | D |
| Jordan Bork      (3)  | Cole Graves      (2)  | 4/7 | Mon | HOU | 6:40pm | D |
| Eric Brechner    (62)  | Al Erisman       (64)  | 4/8 | Tue | HOU | 6:40pm | D |
| Karen Boehmer    (9)  |                       | 4/9 | Wed | HOU | 1:10pm | D |
| Tom Grandine     (9)  | Mike Rochester   (8)  | 4/11 | Fri | TEX | 6:40pm | B |


| | | Date | Day | Opponent | Time | Cost |
| :- | :- | ---: | --- | ------- | ---: | ---- |
| Al Erisman       (4)  | Tom Grandine     (1)  | 3/27 | Thu | ATH | 7:10pm | A |
| Eric Brechner    (11)  | Nick Lederer     (1)  | 3/28 | Fri | ATH | 6:40pm | B |
| Michael Erisman  (40)  |                       | 3/29 | Sat | ATH | 6:40pm | B |
| Cole Graves      (15)  |                       | 3/30 | Sun | ATH | 1:10pm | B |
| Al Erisman       (17)  | Tom Grandine     (50)  | 3/31 | Mon | DET | 6:40pm | D |
| Michael Erisman  (40)  |                       | 4/1 | Tue | DET | 6:40pm | D |
| Eric Brechner    (49)  | Fritz Klein      (10)  | 4/2 | Wed | DET | 1:10pm | D |
| Jordan Bork      (3)  | Cole Graves      (2)  | 4/7 | Mon | HOU | 6:40pm | D |
| Eric Brechner    (62)  | Al Erisman       (64)  | 4/8 | Tue | HOU | 6:40pm | D |
| Karen Boehmer    (9)  |                       | 4/9 | Wed | HOU | 1:10pm | D |
| Tom Grandine     (9)  | Mike Rochester   (8)  | 4/11 | Fri | TEX | 6:40pm | B |
| Cole Graves      (5)  |                       | 4/12 | Sat | TEX | 6:40pm | B |
| Eric Brechner    (33)  | Mike Neff        (23)  | 4/13 | Sun | TEX | 1:10pm | B |
| Eric Brechner    (23)  | Tom Grandine     (7)  | 4/25 | Fri | MIA | 6:40pm | B |
| Ryan Falls       (14)  |                       | 4/26 | Sat | MIA | 6:40pm | B |
| Jordan Bork      (1)  |                       | 4/27 | Sun | MIA | 1:10pm | B |
| Eric Brechner    (32)  | Al Erisman       (72)  | 4/29 | Tue | LAA | 6:40pm | D |
| Karen Boehmer    (7)  | Fritz Klein      (8)  | 4/30 | Wed | LAA | 1:10pm | D |
| Jordan Bork      (6)  |                       | 5/9 | Fri | TOR | 6:40pm | A |
| Tom Grandine     (38)  | Nick Lederer     (3)  | 5/10 | Sat | TOR | 6:40pm | A |
| Eric Brechner    (43)  | Mike Rochester   (3)  | 5/11 | Sun | TOR | 1:10pm | A |
| Mike Neff        (3)  |                       | 5/12 | Mon | NYY | 6:40pm | B |
| Mike Neff        (2)  |                       | 5/13 | Tue | NYY | 6:40pm | B |
| Karen Boehmer    (1)  |                       | 5/14 | Wed | NYY | 1:10pm | B |
| Eric Brechner    (3)  | Michael Erisman  (40)  | 5/27 | Tue | WAS | 6:40pm | D |
| Andrew Erisman   (3)  |                       | 5/28 | Wed | WAS | 6:40pm | D |
| Ryan Falls       (10)  |                       | 5/29 | Thu | WAS | 6:40pm | D |
| Eric Brechner    (67)  | Michael Jones    (8)  | 5/30 | Fri | MIN | 7:10pm | B |
| Andrew Erisman   (4)  |                       | 5/31 | Sat | MIN | 4:15pm | B |
| Jordan Bork      (2)  |                       | 6/1 | Sun | MIN | 1:10pm | B |
| Al Erisman       (24)  | Michael Erisman  (40)  | 6/3 | Tue | BAL | 6:40pm | C |
| Andrew Erisman   (5)  |                       | 6/4 | Wed | BAL | 6:40pm | C |
| Eric Brechner    (57)  | Fritz Klein      (1)  | 6/5 | Thu | BAL | 12:40pm | C |
| Eric Brechner    (7)  | Mike Rochester   (2)  | 6/13 | Fri | CLE | 7:10pm | A |
| Cole Graves      (2)  |                       | 6/14 | Sat | CLE | 6:40pm | A |
| Al Erisman       (5)  | Mike Neff        (22)  | 6/15 | Sun | CLE | 1:10pm | A |
| Andrew Erisman   (1)  |                       | 6/16 | Mon | BOS | 6:40pm | B |
| Ryan Falls       (2)  |                       | 6/17 | Tue | BOS | 6:40pm | B |
| Al Erisman       (8)  | Fritz Klein      (6)  | 6/18 | Wed | BOS | 1:10pm | B |
| Eric Brechner    (37)  | Tom Grandine     (19)  | 6/30 | Mon | KC | 6:40pm | C |
| Al Erisman       (22)  | Michael Erisman  (40)  | 7/1 | Tue | KC | 6:40pm | C |
| Andrew Erisman   (7)  |                       | 7/2 | Wed | KC | 6:40pm | C |
| Fritz Klein      (25)  |                       | 7/3 | Thu | KC | 7:10pm | B |
| Michael Jones    (1)  | Mike Neff        (16)  | 7/4 | Fri | PIT | 1:10pm | A |
| Tom Grandine     (5)  |                       | 7/5 | Sat | PIT | 7:10pm | A |
| Jordan Bork      (5)  | Al Erisman       (10)  | 7/6 | Sun | PIT | 1:10pm | A |
| Eric Brechner    (6)  | Nick Lederer     (6)  | 7/18 | Fri | HOU | 7:10pm | A |
| Ryan Falls       (4)  |                       | 7/19 | Sat | HOU | 6:40pm | A |
| Jordan Bork      (6)  | Mike Neff        (8)  | 7/20 | Sun | HOU | 1:10pm | A |
| Eric Brechner    (56)  | Cole Graves      (1)  | 7/21 | Mon | MIL | 6:40pm | C |
| Al Erisman       (2)  |                       | 7/22 | Tue | MIL | 6:40pm | C |
| Michael Erisman  (40)  | Fritz Klein      (2)  | 7/23 | Wed | MIL | 12:40pm | C |
| Andrew Erisman   (10)  |                       | 7/31 | Thu | TEX | 6:40pm | B |
| Jordan Bork      (14)  | Michael Jones    (4)  | 8/1 | Fri | TEX | 7:10pm | A |
| Karen Boehmer    (5)  |                       | 8/2 | Sat | TEX | 1:10pm | A |
| Cole Graves      (9)  |                       | 8/3 | Sun | TEX | 1:10pm | A |
| Eric Brechner    (54)  | Fritz Klein      (13)  | 8/5 | Tue | CWS | 6:40pm | C |
| Al Erisman       (32)  | Michael Erisman  (40)  | 8/6 | Wed | CWS | 6:40pm | C |
| Andrew Erisman   (12)  |                       | 8/7 | Thu | CWS | 1:10pm | C |
| Tom Grandine     (4)  | Mike Rochester   (6)  | 8/8 | Fri | TB | 6:40pm | A |
| Eric Brechner    (1)  | Mike Neff        (1)  | 8/9 | Sat | TB | 6:40pm | A |
| Karen Boehmer    (6)  |                       | 8/10 | Sun | TB | 1:10pm | A |
| Eric Brechner    (5)  | Mike Rochester   (1)  | 8/22 | Fri | ATH | 7:10pm | B |
| Mike Neff        (9)  |                       | 8/23 | Sat | ATH | 6:40pm | B |
| Mike Neff        (11)  |                       | 8/24 | Sun | ATH | 1:10pm | B |
| Eric Brechner    (10)  | Cole Graves      (5)  | 8/25 | Mon | SD | 6:40pm | B |
| Tom Grandine     (3)  |                       | 8/26 | Tue | SD | 6:40pm | B |
| Al Erisman       (36)  | Fritz Klein      (4)  | 8/27 | Wed | SD | 1:10pm | B |
| Al Erisman       (14)  | Cole Graves      (7)  | 9/8 | Mon | STL | 6:40pm | C |
| Eric Brechner    (15)  | Michael Erisman  (40)  | 9/9 | Tue | STL | 6:40pm | C |
| Andrew Erisman   (16)  |                       | 9/10 | Wed | STL | 6:40pm | C |
| Michael Erisman  (40)  |                       | 9/11 | Thu | LAA | 6:40pm | C |
| Eric Brechner    (4)  | Michael Jones    (6)  | 9/12 | Fri | LAA | 7:10pm | B |
| Tom Grandine     (16)  | Nick Lederer     (13)  | 9/13 | Sat | LAA | 6:40pm | B |
| Fritz Klein      (5)  |                       | 9/14 | Sun | LAA | 1:10pm | B |
| Tom Grandine     (15)  | Fritz Klein      (12)  | 9/23 | Tue | COL | 6:40pm | C |
| Andrew Erisman   (18)  |                       | 9/24 | Wed | COL | 6:40pm | C |
| Eric Brechner    (8)  | Al Erisman       (39)  | 9/25 | Thu | COL | 6:40pm | C |
| Tom Grandine     (2)  | Mike Neff        (6)  | 9/26 | Fri | LAD | 6:40pm | A |
| Cole Graves      (1)  |                       | 9/27 | Sat | LAD | 6:40pm | A |
| Eric Brechner    (12)  | Al Erisman       (13)  | 9/28 | Sun | LAD | 12:10pm | A |Eric Brechner    (12)  | Al Erisman       (13)  | 9/28 | Sun | LAD | 12:10pm | A |ic Brechner    (12)  | Al Erisman       (13)  |  9/28 | Sun | LAD | 12:10pm | A |


In [8]:
print("| | Amount owed | Paid |")
print("| :- | -: | -: |")
for fix, fan in enumerate(fans):
    print("| ", fan.name, " | \$%7.2f | |" % costs[fix])

| | Amount owed | Paid |
| :- | -: | -: |
|  Karen Boehmer     | \$1266.00 | |
|  Jordan Bork       | \$1542.00 | |
|  Eric Brechner     | \$2910.00 | |
|  Al Erisman        | \$2064.00 | |
|  Andrew Erisman    | \$2148.00 | |
|  Michael Erisman   | \$1344.00 | |
|  Ryan Falls        | \$1092.00 | |
|  Tom Grandine      | \$2088.00 | |
|  Cole Graves       | \$2088.00 | |
|  Michael Jones     | \$ 636.00 | |
|  Fritz Klein       | \$1464.00 | |
|  Nick Lederer      | \$ 636.00 | |
|  Mike Neff         | \$2142.00 | |
|  Mike Rochester    | \$ 816.00 | |


| | Amount owed | Paid |
| :- | -: | -: |
|  Karen Boehmer     | \$1266.00 | 3/17/25 |
|  Jordan Bork       | \$1542.00 | 3/23/25 |
|  Eric Brechner     | \$2910.00 | 3/16/25 |
|  Al Erisman        | \$2064.00 | 3/20/25 |
|  Andrew Erisman    | \$2148.00 | 3/19/25 |
|  Michael Erisman   | \$1344.00 | 3/20/25 |
|  Ryan Falls        | \$1092.00 | 3/18/25 |
|  Tom Grandine      | \$2088.00 | 3/16/25 |
|  Cole Graves       | \$2088.00 | 3/24/25 |
|  Michael Jones     | \$ 636.00 | 3/17/25 |
|  Fritz Klein       | \$1464.00 | 3/17/25 |
|  Nick Lederer      | \$ 636.00 | 3/16/25 |
|  Mike Neff         | \$2142.00 | |
|  Mike Rochester    | \$ 816.00 | 3/16/25 |

In [9]:
picks = [[] for fan in fans]
for mix, mvar in enumerate(tixmodel.vars):
    if mvar.x > 0.5:
        fix = mix // (2 * GamesInPlan)
        if len(fans[fix].ranking) > GamesInPlan:
            gix = mix % (2 * GamesInPlan)
        else:
            gix = mix % GamesInPlan
        picks[fix].append(fans[fix].ranking[gix])
for fix, fan in enumerate(fans):
    picks[fix].sort()
    print(fans[fix].name, picks[fix])

Karen Boehmer    [1, 5, 6, 7, 9]
Jordan Bork      [1, 2, 3, 5, 6, 6, 14]
Eric Brechner    [1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 15, 23, 32, 33, 37, 43, 49, 54, 56, 57, 62, 67]
Al Erisman       [2, 4, 5, 8, 10, 13, 14, 17, 22, 24, 32, 36, 39, 64, 72]
Andrew Erisman   [1, 3, 4, 5, 7, 10, 12, 16, 18]
Michael Erisman  [40, 40, 40, 40, 40, 40, 40, 40, 40]
Ryan Falls       [2, 4, 10, 14]
Tom Grandine     [1, 2, 3, 4, 5, 7, 9, 15, 16, 19, 38, 50]
Cole Graves      [1, 1, 2, 2, 5, 5, 7, 9, 15]
Michael Jones    [1, 4, 6, 8]
Fritz Klein      [1, 2, 4, 5, 6, 8, 10, 12, 13, 25]
Nick Lederer     [1, 3, 6, 13]
Mike Neff        [1, 2, 3, 6, 8, 9, 11, 16, 22, 23]
Mike Rochester   [1, 2, 3, 6, 8]


In [10]:
for gix, game in enumerate(schedule):
    picks = []
    if gix < GamesInPlan:
        for fan in fans:
            picks.append(fan.ranking[gix])
    print(picks, "%2d/%2d " % (game.month, game.day) + game.weekday + " " + game.opponent + " " + game.time +
          " " + game.costCode)

[31, 13, 36, 4, 50, 40, 24, 1, 17, 31, 78, 58, 40, 81]  3/27 Thu ATH 7:10pm A
[63, 81, 11, 78, 50, 40, 25, 10, 18, 22, 77, 1, 39, 80]  3/28 Fri ATH 6:40pm B
[64, 80, 81, 79, 50, 40, 26, 11, 56, 23, 76, 5, 74, 79]  3/29 Sat ATH 6:40pm B
[40, 79, 80, 80, 50, 40, 27, 52, 57, 71, 75, 29, 38, 78]  3/30 Sun ATH 1:10pm B
[41, 31, 69, 17, 50, 40, 28, 50, 6, 59, 74, 30, 62, 60]  3/31 Mon DET 6:40pm D
[36, 78, 24, 18, 50, 40, 29, 46, 24, 52, 73, 56, 61, 59]  4/ 1 Tue DET 6:40pm D
[62, 77, 49, 81, 50, 40, 30, 53, 25, 42, 10, 54, 77, 58]  4/ 2 Wed DET 1:10pm D
[42, 3, 68, 63, 50, 40, 31, 47, 2, 58, 64, 53, 55, 40]  4/ 7 Mon HOU 6:40pm D
[37, 76, 62, 64, 50, 40, 32, 48, 29, 51, 63, 52, 52, 39]  4/ 8 Tue HOU 6:40pm D
[9, 75, 61, 65, 50, 40, 33, 54, 30, 41, 9, 51, 76, 23]  4/ 9 Wed HOU 1:10pm D
[45, 4, 35, 66, 50, 40, 11, 9, 19, 24, 62, 31, 27, 8]  4/11 Fri TEX 6:40pm B
[30, 74, 34, 67, 50, 40, 12, 8, 58, 25, 61, 71, 19, 38]  4/12 Sat TEX 6:40pm B
[39, 73, 33, 68, 50, 40, 34, 55, 59, 70, 60, 72, 23, 

In [11]:
GamesInPlan = 81

# Read in the priorities

f = open ("C:/Users/thoma/tom/baseball/2024/tg.txt", 'r')
filedata = f.readlines ()
priority = []
for line in filedata:
  if (len (priority) == GamesInPlan):
    continue
  splitline = line.split ()
  priority.append (float (splitline[0]))
f.close ()

# Renumber the priorities so that consecutive integers are used

nprior = list (range (GamesInPlan))
for ix in range (len (priority)):
  smallest = 1000
  for iy in range (len (priority)):
    if priority[iy] < smallest:
      iz = iy
      smallest = priority[iy]
  nprior[iz] = ix + 1
  priority[iz] = 1000

# Print out the results

print (nprior)


[1, 63, 64, 65, 29, 72, 46, 2, 3, 30, 31, 73, 74, 75, 76, 77, 78, 79, 47, 32, 33, 48, 34, 35, 49, 36, 37, 38, 50, 26, 27, 51, 39, 40, 41, 42, 80, 28, 52, 12, 13, 53, 16, 17, 54, 66, 67, 68, 11, 4, 55, 24, 25, 56, 44, 45, 5, 8, 9, 10, 6, 81, 57, 7, 58, 59, 14, 15, 60, 18, 61, 21, 22, 23, 62, 69, 70, 71, 19, 20, 43]
